In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_fixed
from sklearn.model_selection import ParameterGrid
import pandas as pd
import datetime

def GetParamIfExist(param, params_dict):
    if param in params_dict.keys():
        return params_dict[param]
    else:
        return None

def SaveResults(datasetID, bestScore, bestParams, clfName, model):
    path = "results/ /fixed/{}-{}.csv".format(model, clfName)
    cols = ['model', 'K', 'excludingPercentage', 'clfName', 'distanceMetric', 
            'beta', 'bestMacrof1', 'date']
    
    try:
        results = pd.read_csv(path, index_col=0)
    except FileNotFoundError:
        results = pd.DataFrame(columns = cols)
    
    if (datasetID in  results.index):
        results.loc[datasetID, cols[0]] = model
        results.loc[datasetID, cols[1]] = GetParamIfExist(cols[1], bestParams)
        results.loc[datasetID, cols[2]] = GetParamIfExist(cols[2], bestParams)
        results.loc[datasetID, cols[3]] = GetParamIfExist(cols[3], bestParams)
        results.loc[datasetID, cols[4]] = GetParamIfExist(cols[4], bestParams)
        results.loc[datasetID, cols[5]] = GetParamIfExist(cols[5], bestParams)
        results.loc[datasetID, cols[6]] = bestScore
        results.loc[datasetID, cols[7]] = str(datetime.datetime.now())
    else:
        result = pd.DataFrame([[model,
                                GetParamIfExist(cols[1], bestParams),
                                GetParamIfExist(cols[2], bestParams),
                                GetParamIfExist(cols[3], bestParams),
                                GetParamIfExist(cols[4], bestParams),
                                GetParamIfExist(cols[5], bestParams),
                                bestScore,
                                str(datetime.datetime.now())]],
                                columns=cols, index = [datasetID])
        results = results.append(result)
        
    results.to_csv(path)
    

def writeResults(datasetID, bestScore, bestParams, clfName):
    path = "results/batch/fixed/gridsearch_amanda_fixed_EVL_NOVO-{}.txt".format(clfName)
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadKeystroke, setup.loadNOAADataset]
    
    #datasets = [setup.loadElec2Data]
    #datasets = [setup.loadKeystroke, setup.loadNOAADataset]
    
    arrClfName = ['LP']
    for clfName in arrClfName:
        print("**************** BEGIN of {} results ****************".format(clfName))        
        poolSize = None
        isBatchMode = True
        #testing grid search
        
        for i in range(len(datasets)):
            splitPercentage = 0.05
            batches = 10

            if i==len(datasets)-2: #Keystroke
                batches=8
            elif i==len(datasets)-1: #NOAA
                batches=4

            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(splitPercentage*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            initialLabeledData = int((1/batches)*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/(batches - 1))

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"excludingPercentage" : [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5],
                             "sizeOfBatch":[sizeOfBatch], "batches":[batches-1], "poolSize":[poolSize],
                             "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData], "clfName":[clfName]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_fixed.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except:
                    print("An error occured in ", description, g)
                    #raise

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            #writeResults(description, finalScore, best_grid, clfName)
            SaveResults(description, finalScore, best_grid, clfName, 'amandafcp')
        print("******** END of {} results ********".format(clfName))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP results ****************
One Class Diagonal Translation. 2 Dimensional data.: 10 batches of 80 instances
0.9608672149008657 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9650447863488597 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9608459725137681 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9581736913531148 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9623751143729731 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9651528

0.9494015136856405 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9518374603337862 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.65, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9735066488734295 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9651626943449136 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.902790133480416 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8407565754194288 {'K': 8, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize'

0.9604751594449995 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9491207145946452 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9790294233671355
{'K': 5, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
One Class Horizontal Translation. 2 Dimensional data.: 10 batches of 80 instances
0.8824379491563817 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9063514394465978 {'K': 2, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9052563090964161 {'K': 2, 'batches': 9, 'clfName': 'LP', 

0.8814300908679169 {'K': 6, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.6987630763245049 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8313037573784947 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8622169177872221 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8937634713803271 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9031187307247376 {'K': 7, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize

0.6769386407468119 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.6913650218204003 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.789416761359549 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.65, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8401130296542596 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.7829836553751159 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.7712579057768325 {'K': 12, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 80, 'isBatchMode': True, 'po

...

0.6904558692063841 {'K': 5, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6723497648063989 {'K': 5, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6869023536913436 {'K': 5, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6716831049102917 {'K': 6, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6572206510274521 {'K': 6, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6753900722512028 {'K': 6, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 137, 'isBatchMode': True

0.5819384984480356 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.5974508477111848 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6387771494187039 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6849338073160879 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.65, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6680554686697402 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 137, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 137}
0.6429561585078283 {'K': 11, 'batches': 9, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 137, 'isBatchMode

0.30986632415203846 {'K': 5, 'batches': 7, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 10, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 10}
0.0918803418803419 {'K': 6, 'batches': 7, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 10, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 10}
An error occured in  Keyboard patterns database. 10 features. 4 classes. {'K': 6, 'batches': 7, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 10, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 10}
An error occured in  Keyboard patterns database. 10 features. 4 classes. {'K': 6, 'batches': 7, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 10, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 10}
0.09249084249084251 {'K': 6, 'batches': 7, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 10, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 10}
0.12701465201465204 {'K': 6, 'bat

0.6433908255338089 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.604344241705912 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.576564036154697 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.65, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.5713890906389932 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.5566567240741062 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.5930441493525197 {'K': 2, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 226, 'isBatchMode': True,

0.5457283777926555 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.429188395976631 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.8, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.45649687436034564 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.75, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.48354499351582625 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.7, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.47513299249972224 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.65, 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.44223695074868 {'K': 8, 'batches': 3, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 226, 'isBatchMode': Tru